!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="test", use_auth_token=True)

print(common_voice)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 36125
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10143
    })
})


In [3]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 36125
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10143
    })
})


In [4]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [5]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

In [6]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

In [7]:
print(common_voice["train"][0])

{'audio': {'path': '/home/ec2-user/.cache/huggingface/datasets/downloads/extracted/3d19a9c51af3bc20a942812b14f5c8c5e9ffeedfae87fd6ebb1e22f3f5e3f112/tr_train_0/common_voice_tr_28856093.mp3', 'array': array([ 1.42108547e-14, -4.97379915e-14, -1.84741111e-13, ...,
       -9.25478844e-07,  6.68777648e-06,  5.80005371e-06]), 'sampling_rate': 48000}, 'sentence': 'Sadece birkaç gün.'}


In [8]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [9]:
print(common_voice["train"][0])

{'audio': {'path': '/home/ec2-user/.cache/huggingface/datasets/downloads/extracted/3d19a9c51af3bc20a942812b14f5c8c5e9ffeedfae87fd6ebb1e22f3f5e3f112/tr_train_0/common_voice_tr_28856093.mp3', 'array': array([-1.45519152e-11, -9.09494702e-12, -3.63797881e-12, ...,
        1.13121750e-05, -1.24729445e-06, -1.83326893e-06]), 'sampling_rate': 16000}, 'sentence': 'Sadece birkaç gün.'}


In [10]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [11]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"],num_proc=4)

Map (num_proc=4):   0%|          | 0/36125 [00:00<?, ? examples/s]

Exception ignored from cffi callback <function SoundFile._init_virtual_io.<locals>.vio_read at 0x7f346556c3a0>:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/soundfile.py", line 1245, in vio_read
    buf = _ffi.buffer(ptr, count)
KeyboardInterrupt: 


TimeoutError: 

In [ ]:
help(common_voice)

In [ ]:
common_voice.save_to_disk("dataset")

In [ ]:
#common_voice.save_to_disk("s3://project-whisper-data/dataset")